In [3]:
import random
import shelve

from twnews.utils.memoize import load
from twnews.dataset.storage import TweetsStorage
from twnews.dataset.url_resolver import UrlResolver
from twnews.dataset.texts import Tweet
from twnews import defaults
from twnews.utils.text_processors import Lemmatizer
from twnews.resolver import get_domain

#sample_size = 1000000

In [4]:
corpus, _  = load('tf_idf_corpus')
corpus = set(corpus)
lemmatizer = Lemmatizer()
data_shelve = shelve.open(defaults.TWEETS_PATH)
resolver = UrlResolver()

2016-05-07 09:51:49.006339: Start of loading url_map from /home/avybornov/twnews_data_april/resolve_url_map.shelve
2016-05-07 09:51:49.006689: Url map successfully loaded


In [6]:
keys = data_shelve.keys()
random.shuffle(keys)

tweets = []
for key in keys[:100]:
    tweet = Tweet(data_shelve[key])
    tweet.resolve_urls(resolver)
    tweets.append(tweet)

In [149]:
def print_tweet(tweet):
    print tweet.text.replace('\n',' ')
    
    if tweet.retweet:
        print '\t', 'retweet'
    if tweet.urls:
        for url in tweet.urls:
            print '\t url: ', url
    for h in tweet.hastags.keys():
        print '\t hashtag:', h
    
    

In [150]:
min_num_of_words = 5
min_num_of_words_in_corpus = 2
bad_domain = set(['apps.facebook.com', 'ask.fm', 'twitter.com', 'apps.facebook.com', 'www.instagram.com', 'www.instagram.com', 'vk.cc'])

def is_tweet_good(tweet):
    if tweet.retweet:
        return False
    
    for url in tweet.urls:
        domain = get_domain(url)
        if domain in bad_domain:
            return False
        
    text = tweet.text
    
    for h in tweet.hastags:
        text = text.replace(h, '')
    
    words = lemmatizer.split_text_to_lemmas(text)
    if len(words) < min_num_of_words:
        return False
    
    num_of_words_in_corpus = sum(map(lambda x: x in corpus, words))
    if num_of_words_in_corpus < min_num_of_words:
        return False
    tweet.num_of_words_in_corpus = num_of_words_in_corpus
    
    return True


# for i,tweet in enumerate(tweets[:1000]):
#     print i, '------------------'
#     good_tweet = is_tweet_good(tweet)
#     print good_tweet,
#     print_tweet(tweet)
#     if good_tweet:
#         print '\t words in corpus: ', tweet.num_of_words_in_corpus



In [151]:
def filtration(tweets):
    res = []
    for i, tweet in enumerate(tweets):
        if is_tweet_good(tweet):
            res.append(tweet)
        if i % 5000 == 0:
            print '%.2f%%' % (i*100.0/len(tweets))
    print len(tweets), len(res)
    return res
#good_tweets = filter(is_tweet_good, tweets)

In [1]:
№good_tweets = filtration(tweets)

SyntaxError: invalid syntax (<ipython-input-1-5d0289d263b4>, line 1)

In [154]:
from twnews.utils.memoize import dump
dump(good_tweets, 'good_tweets')

In [16]:
#import codecs



tweet_texts = []
for tweet in tweets[:1000]:
    tweet_texts.append(tweet.text.replace('\n', ' ').replace('.',';'))
text = '. '.join(tweet_texts)


In [29]:
tweet_NE = []
# print len(tweets)

# polyglot_text = Text(text)
# for sent in polyglot_text.sentences:
# #    print(sent, "\n")
#     tweet_NE.append((sent, sent.entities))
# print len(tweet_NE)

for tweet in tweets[:1000]:
    #print tweet.text
    t = Text(tweet.text)
    t.language = 'ru'
    ent = t.entities
    tweet_NE.append((tweet, ent))
    #print(entity.tag, entity)
    

In [45]:
for t,e in tweet_NE:
    print t.text.replace('\n',' ')
    s = ' '.join(lemmatizer.split_text_to_lemmas(t.text))
    print s
    s = Text(s)
    s.language = 'ru'
    #for x in s.entities:
        #print '\t', x.tag, x[0]
    print len(e), len(s.entities)
    print '---------------'

Правильно ли поступил мэр Лужков, запретив Русский марш 4 ноября?
правильно поступить мэр лужков запретить русский марш ноябрь
1 0
---------------
@yanix_official можно ты пойдешь нахуй и поедешь куда угодно тока не в хабаровск блять
yanix_official пойти нахуй поехать угодный ток хабаровск блять
1 1
---------------
Ребята, корейский учить тяжелооо
ребята корейский учить тяжелооо
0 0
---------------
хочу руссифицированую аську
хотеть руссифицированый аська
0 1
---------------
Мужская и женская коллекция со скидкой до 70%! | KupiVIP 
мужской женский коллекция скидка KupiVIP
0 0
---------------
@russlan_v но 50 ему тоже было, и это увековечено на фотках и в кинематографе!
russlan_v это увековечить фотка кинематограф
0 0
---------------
Пошла Машенька по грибы, да по ягоды. Вернулась ни с чем. Потому что надо ставить перед собой конкретные цели
пошлый машенька гриб ягода вернуться ставить конкретный цель
0 0
---------------
Сначала кидают, а потом общаются как будто нечего не было
сначала 